# **Chains and LangChain Expression Language (LCEL)**

## **Learning Objectives**
By the end of this notebook, you will be able to:
- Build chains using the pipe operator (|) in LCEL
- Connect multiple components into complex workflows
- Implement parallel and sequential execution patterns
- Use output parsers to structure LLM responses
- Handle errors and implement fallbacks in chains
- Create reusable chain components

## **Why This Matters: Building Production Workflows**

**In Real Applications:**
- Chains connect prompts → LLMs → parsers → tools
- LCEL enables readable, maintainable pipelines
- Complex workflows become simple to express

**In RAG Systems:**
- Query → Retrieval → Context → Generation → Formatting
- All connected seamlessly with LCEL
- Easy to modify and extend pipelines

**In AI Agents:**
- Tool selection → Execution → Response parsing
- Decision trees and conditional logic
- Parallel tool execution for efficiency

## **Prerequisites**
- Completed notebooks 00, 01, and 02
- Understanding of prompts and templates
- Basic knowledge of function composition

## **Setup: Install and Import Dependencies**

Run this cell first to set up your environment:

In [ ]:
# Install required packages
!pip install -q langchain langchain-openai python-dotenv

# Import necessary modules
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import (
    StrOutputParser,
    JsonOutputParser,
    PydanticOutputParser,
    CommaSeparatedListOutputParser
)
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from pydantic import BaseModel, Field
from typing import List, Dict

# Load environment variables
load_dotenv()

# Verify setup
if os.getenv("OPENAI_API_KEY"):
    print("✅ Environment ready! Let's build chains with LCEL.")
else:
    print("⚠️ Please set your OPENAI_API_KEY")

---

## **Instructor Activity 1: Introduction to LCEL and the Pipe Operator**

**Concept**: LCEL uses the pipe operator (|) to chain components together, creating readable workflows that process data step by step.

### **Example 1: Your First Chain**

**Problem**: Connect prompt → LLM → parser in a single chain
**Expected Output**: A complete processing pipeline

In [ ]:
# Empty cell for demonstration

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# **Define components**
prompt = ChatPromptTemplate.from_template(
    "Tell me a {adjective} joke about {topic}."
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.8)

output_parser = StrOutputParser()

# **Chain components together with the pipe operator**
chain = prompt | llm | output_parser

# **The chain flows: prompt → llm → parser**
# **Input dict → formatted prompt → LLM response → parsed string**

# **Run the chain**
result = chain.invoke({
    "adjective": "funny",
    "topic": "programming"
})

print("Chain Execution:")
print("=" * 50)
print("Input → Prompt → LLM → Parser → Output")
print("=" * 50)
print(result)

# **You can also see intermediate steps**
print("\n📊 Chain Structure:")
print(f"1. Prompt Template: Takes {prompt.input_variables}")
print(f"2. LLM: {llm.model_name}")
print(f"3. Parser: {output_parser.__class__.__name__}")
```

**Why LCEL is powerful:**
- Clean, readable syntax
- Components are composable
- Easy to modify pipelines
- Type-safe data flow

</details>

### **Example 2: Understanding Data Flow**

**Problem**: Visualize how data transforms through the chain
**Expected Output**: Clear understanding of each step

In [ ]:
# Empty cell for demonstration

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# **Create a chain with logging at each step**
def create_debug_chain():
    """Create a chain that shows data at each step"""
    
    # Step 1: Prompt Template
    prompt = ChatPromptTemplate.from_template(
        "Translate '{text}' to {language}."
    )
    
    # Step 2: LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # Step 3: Output Parser
    parser = StrOutputParser()
    
    # Build the chain
    chain = prompt | llm | parser
    
    return chain, prompt, llm, parser

# **Create and inspect the chain**
chain, prompt, llm, parser = create_debug_chain()

# **Input data**
input_data = {
    "text": "Hello, how are you?",
    "language": "Spanish"
}

print("🔍 Data Flow Through Chain:")
print("=" * 50)

# **Step 1: Show formatted prompt**
formatted_prompt = prompt.format_messages(**input_data)
print("Step 1 - Formatted Prompt:")
for msg in formatted_prompt:
    print(f"  {msg.__class__.__name__}: {msg.content}")

# **Step 2: LLM processes the prompt**
llm_response = llm.invoke(formatted_prompt)
print(f"\nStep 2 - LLM Response (type: {type(llm_response).__name__}):")
print(f"  Content: {llm_response.content}")
print(f"  Metadata: {llm_response.response_metadata.get('model_name', 'N/A')}")

# **Step 3: Parser extracts content**
parsed_output = parser.invoke(llm_response)
print(f"\nStep 3 - Parsed Output (type: {type(parsed_output).__name__}):")
print(f"  {parsed_output}")

# **Now run the complete chain**
print("\n" + "=" * 50)
print("Complete Chain Execution:")
final_result = chain.invoke(input_data)
print(f"Result: {final_result}")

print("\n💡 Each component transforms the data for the next step!")
```

**Data transformation flow:**
1. Dict → ChatPromptTemplate → Messages
2. Messages → LLM → AIMessage
3. AIMessage → Parser → String

</details>

### **Example 3: Chaining Multiple Operations**

**Problem**: Build a chain that performs multiple transformations
**Expected Output**: Complex multi-step pipeline

In [ ]:
# Empty cell for demonstration

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# **Create a multi-step chain**
# **Step 1: Generate a story**
story_prompt = ChatPromptTemplate.from_template(
    "Write a 2-sentence story about {character} in {setting}."
)

# **Step 2: Extract the moral**
moral_prompt = ChatPromptTemplate.from_template(
    "What is the moral of this story: {story}"
)

# **Step 3: Simplify for children**
simplify_prompt = ChatPromptTemplate.from_template(
    "Explain this moral to a 5-year-old: {moral}"
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.8)
parser = StrOutputParser()

# **Build the complete chain**
complete_chain = (
    # Generate story
    story_prompt 
    | llm 
    | parser
    # Pass story to next prompt
    | RunnableLambda(lambda x: {"story": x})
    | moral_prompt
    | llm
    | parser
    # Pass moral to simplification
    | RunnableLambda(lambda x: {"moral": x})
    | simplify_prompt
    | llm
    | parser
)

# **You can also build it step by step for clarity**
story_chain = story_prompt | llm | parser
moral_chain = moral_prompt | llm | parser
simplify_chain = simplify_prompt | llm | parser

# **Manual step-by-step execution**
print("Step-by-Step Chain Execution:")
print("=" * 50)

input_data = {"character": "a wise owl", "setting": "a magical forest"}

# **Step 1**
story = story_chain.invoke(input_data)
print("📖 Story:")
print(story)

# **Step 2**
moral = moral_chain.invoke({"story": story})
print("\n💭 Moral:")
print(moral)

# **Step 3**
simple_moral = simplify_chain.invoke({"moral": moral})
print("\n👶 For Kids:")
print(simple_moral)

# **Now run the complete chain**
print("\n" + "=" * 50)
print("Complete Chain Result:")
result = complete_chain.invoke(input_data)
print(result)

print("\n✅ Chains can have multiple transformation steps!")
```

**Multi-step chain benefits:**
- Each step builds on the previous
- Clear data flow
- Easy to debug individual steps
- Modular and reusable

</details>

---

## **Learner Activity 1: Practice Building Chains**

**Practice Focus**: Create your own chains using LCEL

### **Exercise 1: Build a Translation Chain**

**Task**: Create a chain that translates text to multiple languages
**Expected Output**: Multi-language translation

In [ ]:
# Your code here
# TODO: Create a chain that:
# 1. Takes input text
# 2. Translates to Spanish
# 3. Then translates the Spanish to French
# Use prompt | llm | parser for each step

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# **Initialize components**
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

# **Create translation prompts**
to_spanish = ChatPromptTemplate.from_template(
    "Translate this English text to Spanish: {text}"
)

to_french = ChatPromptTemplate.from_template(
    "Translate this Spanish text to French: {spanish_text}"
)

# **Build the translation chain**
translation_chain = (
    to_spanish
    | llm
    | parser
    | RunnableLambda(lambda x: {"spanish_text": x})
    | to_french
    | llm
    | parser
)

# **Test the chain**
original_text = "Good morning! How are you today?"

print("Translation Chain:")
print("=" * 50)
print(f"🇬🇧 English: {original_text}")

# **Get Spanish translation first (for demonstration)**
spanish = (to_spanish | llm | parser).invoke({"text": original_text})
print(f"🇪🇸 Spanish: {spanish}")

# **Get final French translation**
french = translation_chain.invoke({"text": original_text})
print(f"🇫🇷 French: {french}")

print("\n✅ Chain successfully translated through multiple languages!")
```

**What you learned:**
- Chaining multiple translation steps
- Using RunnableLambda for data transformation
- Sequential processing with LCEL

</details>

### **Exercise 2: Create a Summary Chain**

**Task**: Build a chain that summarizes text and extracts key points
**Expected Output**: Summary and bullet points

In [ ]:
# Your code here
# TODO: Create a chain that:
# 1. Summarizes a long text
# 2. Extracts 3 key points
# 3. Formats as bullet points

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# **Initialize LLM**
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

# **Create prompts for each step**
summarize_prompt = ChatPromptTemplate.from_template(
    """Summarize this text in 2-3 sentences:
    
    {text}
    
    Summary:"""
)

extract_points_prompt = ChatPromptTemplate.from_template(
    """From this summary, extract exactly 3 key points as bullet points:
    
    {summary}
    
    Key Points:"""
)

# **Build the summary chain**
summary_chain = (
    summarize_prompt
    | llm
    | parser
    | (lambda x: {"summary": x})
    | extract_points_prompt
    | llm
    | parser
)

# **Test with sample text**
long_text = """Artificial Intelligence (AI) has transformed how we interact with technology. 
From voice assistants like Siri and Alexa to recommendation systems on Netflix and Spotify, 
AI is everywhere. Machine learning models can now diagnose diseases, drive cars, and even 
create art. However, with great power comes great responsibility. We must consider the 
ethical implications of AI, including privacy concerns, job displacement, and algorithmic bias. 
The future of AI promises even more advances, with potential breakthroughs in general 
artificial intelligence that could revolutionize every aspect of human life."""

print("Summary Chain Results:")
print("=" * 50)
print("📄 Original Text:")
print(long_text[:150] + "...\n")

# **Get summary only**
summary = (summarize_prompt | llm | parser).invoke({"text": long_text})
print("📝 Summary:")
print(summary)

# **Get key points**
key_points = summary_chain.invoke({"text": long_text})
print("\n🎯 Key Points:")
print(key_points)

print("\n✅ Chain extracted summary and key points successfully!")
```

**Key takeaway:**
- Chains can progressively refine information
- Each step focuses on a specific task
- Output becomes more structured

</details>

---

## **Instructor Activity 2: Output Parsing in Chains**

**Concept**: Output parsers convert LLM responses into structured data formats that can be used by applications.

### **Example 1: JSON Output Parser**

**Problem**: Extract structured JSON data from LLM responses
**Expected Output**: Python dictionaries from text

In [ ]:
# Empty cell for demonstration

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# **Define the structure we want**
class Product(BaseModel):
    name: str = Field(description="Product name")
    price: float = Field(description="Price in dollars")
    category: str = Field(description="Product category")
    in_stock: bool = Field(description="Whether item is in stock")
    rating: float = Field(description="Customer rating out of 5")

# **Create JSON parser with schema**
json_parser = JsonOutputParser(pydantic_object=Product)

# **Create prompt with format instructions**
prompt = ChatPromptTemplate.from_template(
    """Extract product information from this description:
    
    {description}
    
    {format_instructions}
    """
)

# **Build the chain**
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
json_chain = prompt | llm | json_parser

# **Test with product descriptions**
descriptions = [
    "The SuperWidget 3000 is our best-selling gadget at $49.99. It's in the electronics category, currently in stock, with a 4.5 star rating.",
    "Get the ComfyPillow Plus for just $29.99! This home goods item is flying off our shelves (in stock!) with an amazing 4.8 star customer rating."
]

print("JSON Output Parsing:")
print("=" * 50)

for desc in descriptions:
    result = json_chain.invoke({
        "description": desc,
        "format_instructions": json_parser.get_format_instructions()
    })
    
    print(f"\n📝 Description: {desc[:50]}...")
    print(f"📊 Parsed Data (type: {type(result).__name__}):")
    for key, value in result.items():
        print(f"  {key}: {value}")

print("\n✅ JSON parser extracts structured data automatically!")
```

**JSON parsing benefits:**
- Structured data extraction
- Type validation
- Direct use in applications
- Consistent format

</details>

### **Example 2: Pydantic Output Parser**

**Problem**: Get type-safe, validated data from LLM
**Expected Output**: Pydantic model instances

In [ ]:
# Empty cell for demonstration

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
from datetime import datetime

# **Define complex Pydantic model with validation**
class Meeting(BaseModel):
    title: str = Field(description="Meeting title")
    date: str = Field(description="Meeting date in YYYY-MM-DD format")
    time: str = Field(description="Meeting time in HH:MM format")
    duration_minutes: int = Field(description="Duration in minutes")
    attendees: List[str] = Field(description="List of attendee names")
    agenda_items: List[str] = Field(description="List of agenda items")
    location: str = Field(description="Meeting location or 'Virtual'")
    
    @validator('duration_minutes')
    def duration_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Duration must be positive')
        return v
    
    @validator('attendees')
    def at_least_two_attendees(cls, v):
        if len(v) < 2:
            raise ValueError('Need at least 2 attendees')
        return v

# **Create Pydantic parser**
pydantic_parser = PydanticOutputParser(pydantic_object=Meeting)

# **Create prompt**
prompt = ChatPromptTemplate.from_template(
    """Extract meeting details from this email:
    
    {email}
    
    {format_instructions}
    """
)

# **Build chain**
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
meeting_chain = prompt | llm | pydantic_parser

# **Test email**
email = """Hi team,

Let's have our quarterly review meeting on 2024-03-15 at 14:00. 
The meeting will last about 90 minutes and will be held in Conference Room A.

Attendees: John Smith, Sarah Johnson, Mike Chen, Lisa Anderson

Agenda:
1. Q1 Performance Review
2. Budget Updates
3. New Project Proposals
4. Team Feedback

See you there!
"""

# **Parse the email**
result = meeting_chain.invoke({
    "email": email,
    "format_instructions": pydantic_parser.get_format_instructions()
})

print("Pydantic Output Parsing:")
print("=" * 50)
print(f"✅ Parsed Meeting (type: {type(result).__name__}):")
print(f"  Title: {result.title}")
print(f"  Date: {result.date}")
print(f"  Time: {result.time}")
print(f"  Duration: {result.duration_minutes} minutes")
print(f"  Location: {result.location}")
print(f"  Attendees: {', '.join(result.attendees)}")
print(f"  Agenda:")
for item in result.agenda_items:
    print(f"    - {item}")

# **Demonstrate validation**
print("\n🔍 Validation Check:")
print(f"  Attendee count: {len(result.attendees)} (✓ >= 2)")
print(f"  Duration: {result.duration_minutes} min (✓ > 0)")

print("\n💡 Pydantic provides type safety and validation!")
```

**Pydantic parser advantages:**
- Type safety with validation
- Complex nested structures
- Custom validation rules
- IDE autocomplete support

</details>

### **Example 3: List and Custom Parsers**

**Problem**: Parse different output formats
**Expected Output**: Lists and custom formats

In [ ]:
# Empty cell for demonstration

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import (
    CommaSeparatedListOutputParser,
    StructuredOutputParser,
    ResponseSchema
)

# **1. Comma-separated list parser**
list_parser = CommaSeparatedListOutputParser()

list_prompt = ChatPromptTemplate.from_template(
    """List 5 {category} items.
    {format_instructions}
    """
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

list_chain = list_prompt | llm | list_parser

# **Test list parser**
categories = ["programming languages", "fruits", "countries in Europe"]

print("List Output Parsing:")
print("=" * 50)

for category in categories:
    result = list_chain.invoke({
        "category": category,
        "format_instructions": list_parser.get_format_instructions()
    })
    print(f"\n📝 {category.title()}:")
    for i, item in enumerate(result, 1):
        print(f"  {i}. {item}")

# **2. Structured output parser with schemas**
response_schemas = [
    ResponseSchema(name="summary", description="One sentence summary"),
    ResponseSchema(name="pros", description="List of pros, separated by semicolons"),
    ResponseSchema(name="cons", description="List of cons, separated by semicolons"),
    ResponseSchema(name="rating", description="Rating from 1-10")
]

structured_parser = StructuredOutputParser.from_response_schemas(response_schemas)

review_prompt = ChatPromptTemplate.from_template(
    """Review this product: {product}
    
    {format_instructions}
    """
)

review_chain = review_prompt | llm | structured_parser

# **Test structured parser**
print("\n\nStructured Output Parsing:")
print("=" * 50)

product = "AI-powered smart home assistant"
review = review_chain.invoke({
    "product": product,
    "format_instructions": structured_parser.get_format_instructions()
})

print(f"\n🛍️ Product Review: {product}")
print(f"📝 Summary: {review['summary']}")
print(f"✅ Pros: {review['pros']}")
print(f"❌ Cons: {review['cons']}")
print(f"⭐ Rating: {review['rating']}/10")

print("\n✅ Different parsers for different output needs!")
```

**Parser selection guide:**
- CommaSeparatedListOutputParser: Simple lists
- StructuredOutputParser: Key-value pairs
- JsonOutputParser: Complex nested data
- PydanticOutputParser: Type-safe with validation

</details>

---

## **Learner Activity 2: Practice Output Parsing**

**Practice Focus**: Use different parsers in your chains

### **Exercise 1: Extract Event Information**

**Task**: Create a chain that extracts event details as JSON
**Expected Output**: Structured event data

In [ ]:
# Your code here
# TODO: Create a Pydantic model for an Event with:
# - name, date, time, venue, ticket_price
# Build a chain that extracts this from text descriptions

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# **Define Event model**
class Event(BaseModel):
    name: str = Field(description="Event name")
    date: str = Field(description="Event date")
    time: str = Field(description="Event start time")
    venue: str = Field(description="Event venue/location")
    ticket_price: float = Field(description="Ticket price in dollars")

# **Create parser and prompt**
event_parser = PydanticOutputParser(pydantic_object=Event)

event_prompt = ChatPromptTemplate.from_template(
    """Extract event information from this announcement:
    
    {announcement}
    
    {format_instructions}
    """
)

# **Build chain**
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
event_chain = event_prompt | llm | event_parser

# **Test with event announcements**
announcements = [
    """Join us for the Annual Tech Conference on March 20th, 2024 at 9:00 AM. 
    The event will be held at the Convention Center. Tickets are $150 each.""",
    
    """Don't miss the Summer Music Festival! Saturday, June 15th starting at 2 PM 
    at Central Park. Early bird tickets just $45!"""
]

print("Event Information Extraction:")
print("=" * 50)

for announcement in announcements:
    event = event_chain.invoke({
        "announcement": announcement,
        "format_instructions": event_parser.get_format_instructions()
    })
    
    print(f"\n📅 Event Details:")
    print(f"  Name: {event.name}")
    print(f"  Date: {event.date}")
    print(f"  Time: {event.time}")
    print(f"  Venue: {event.venue}")
    print(f"  Price: ${event.ticket_price}")

print("\n✅ Successfully extracted structured event data!")
```

**What you learned:**
- Defining Pydantic models for data
- Extracting structured information
- Type-safe parsing in chains

</details>

---

## **Instructor Activity 3: Parallel and Advanced Chains**

**Concept**: LCEL supports parallel execution and complex routing patterns for efficient workflows.

### **Example 1: Parallel Execution**

**Problem**: Run multiple chains simultaneously
**Expected Output**: Parallel results

In [ ]:
# Empty cell for demonstration

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import time

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
parser = StrOutputParser()

# **Create different analysis prompts**
sentiment_prompt = ChatPromptTemplate.from_template(
    "Analyze the sentiment of this text (positive/negative/neutral): {text}"
)

summary_prompt = ChatPromptTemplate.from_template(
    "Summarize this text in one sentence: {text}"
)

keywords_prompt = ChatPromptTemplate.from_template(
    "Extract 3 main keywords from this text: {text}"
)

language_prompt = ChatPromptTemplate.from_template(
    "What language is this text written in: {text}"
)

# **Create individual chains**
sentiment_chain = sentiment_prompt | llm | parser
summary_chain = summary_prompt | llm | parser
keywords_chain = keywords_prompt | llm | parser
language_chain = language_prompt | llm | parser

# **Create parallel chain**
parallel_chain = RunnableParallel(
    sentiment=sentiment_chain,
    summary=summary_chain,
    keywords=keywords_chain,
    language=language_chain,
    original=RunnablePassthrough()  # Pass through original text
)

# **Test text**
text = """LangChain is an amazing framework for building AI applications. 
It makes working with large language models so much easier and more enjoyable. 
The community is growing rapidly and the documentation is excellent."""

print("Parallel Chain Execution:")
print("=" * 50)
print("Analyzing text in parallel...\n")

# **Time the parallel execution**
start_time = time.time()
results = parallel_chain.invoke({"text": text})
parallel_time = time.time() - start_time

# **Display results**
print("📊 Analysis Results:")
print(f"  Sentiment: {results['sentiment']}")
print(f"  Summary: {results['summary']}")
print(f"  Keywords: {results['keywords']}")
print(f"  Language: {results['language']}")
print(f"\n⏱️ Parallel execution time: {parallel_time:.2f}s")

# **Compare with sequential execution**
print("\nComparing with sequential execution...")
start_time = time.time()
sentiment = sentiment_chain.invoke({"text": text})
summary = summary_chain.invoke({"text": text})
keywords = keywords_chain.invoke({"text": text})
language = language_chain.invoke({"text": text})
sequential_time = time.time() - start_time

print(f"⏱️ Sequential execution time: {sequential_time:.2f}s")
print(f"\n🚀 Parallel is ~{sequential_time/parallel_time:.1f}x faster!")

print("\n✅ Parallel execution runs multiple chains simultaneously!")
```

**Parallel execution benefits:**
- Significant speed improvement
- Better resource utilization
- All results at once
- Clean syntax with RunnableParallel

</details>

### **Example 2: Conditional Routing**

**Problem**: Route to different chains based on input
**Expected Output**: Dynamic chain selection

In [ ]:
# Empty cell for demonstration

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableBranch

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
parser = StrOutputParser()

# **Create specialized prompts for different content types**
technical_prompt = ChatPromptTemplate.from_template(
    """You are a technical expert. Provide a detailed technical explanation of: {input}
    Include technical terms and implementation details."""
)

simple_prompt = ChatPromptTemplate.from_template(
    """You are explaining to a beginner. Explain this in very simple terms: {input}
    Use analogies and avoid technical jargon."""
)

creative_prompt = ChatPromptTemplate.from_template(
    """You are a creative writer. Write a creative story or poem about: {input}
    Be imaginative and entertaining."""
)

# **Create classifier to determine content type**
classifier_prompt = ChatPromptTemplate.from_template(
    """Classify this query into one category: 'technical', 'simple', or 'creative'.
    Query: {input}
    Return only the category word."""
)

classifier_chain = classifier_prompt | llm | parser | (lambda x: x.strip().lower())

# **Create specialized chains**
technical_chain = technical_prompt | llm | parser
simple_chain = simple_prompt | llm | parser
creative_chain = creative_prompt | llm | parser

# **Create routing function**
def route_query(input_dict):
    """Route to appropriate chain based on classification"""
    classification = classifier_chain.invoke(input_dict)
    
    print(f"🎯 Classified as: {classification}")
    
    if "technical" in classification:
        return technical_chain.invoke(input_dict)
    elif "simple" in classification:
        return simple_chain.invoke(input_dict)
    elif "creative" in classification:
        return creative_chain.invoke(input_dict)
    else:
        return simple_chain.invoke(input_dict)  # Default

# **Create the routing chain**
routing_chain = RunnableLambda(route_query)

# **Test with different queries**
test_queries = [
    "How does a neural network backpropagation algorithm work?",
    "What is a computer?",
    "Write a poem about artificial intelligence"
]

print("Conditional Routing Chain:")
print("=" * 50)

for query in test_queries:
    print(f"\n📝 Query: {query}")
    response = routing_chain.invoke({"input": query})
    print(f"📤 Response: {response[:200]}...\n")
    print("-" * 30)

print("\n✅ Routing enables dynamic chain selection!")
```

**Routing benefits:**
- Different handling for different inputs
- Specialized processing pipelines
- Dynamic workflow adaptation
- Optimal response generation

</details>

---

## **Learner Activity 3: Build Advanced Chains**

**Practice Focus**: Create parallel and conditional chains

### **Exercise 1: Build a Parallel Analysis Chain**

**Task**: Create a chain that analyzes text in multiple ways simultaneously
**Expected Output**: Multiple analysis results

In [ ]:
# Your code here
# TODO: Create a parallel chain that:
# 1. Counts words
# 2. Identifies the topic
# 3. Detects the tone
# Run all three in parallel

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

# **Create analysis prompts**
word_count_prompt = ChatPromptTemplate.from_template(
    "Count the number of words in this text and return just the number: {text}"
)

topic_prompt = ChatPromptTemplate.from_template(
    "What is the main topic of this text? Answer in 2-3 words: {text}"
)

tone_prompt = ChatPromptTemplate.from_template(
    "What is the tone of this text (formal/informal/neutral/excited/serious)?: {text}"
)

# **Create parallel analysis chain**
analysis_chain = RunnableParallel(
    word_count=word_count_prompt | llm | parser,
    topic=topic_prompt | llm | parser,
    tone=tone_prompt | llm | parser,
    # Bonus: add a simple character count
    char_count=RunnableLambda(lambda x: str(len(x['text'])))
)

# **Test texts**
texts = [
    """Machine learning is revolutionizing how we solve problems. 
    From healthcare to finance, AI is making a huge impact!""",
    
    """The quarterly report shows concerning trends. Revenue is down 15% 
    and we need to take immediate action to address these challenges."""
]

print("Parallel Text Analysis:")
print("=" * 50)

for i, text in enumerate(texts, 1):
    print(f"\n📄 Text {i}: {text[:50]}...")
    
    results = analysis_chain.invoke({"text": text})
    
    print("📊 Analysis Results:")
    print(f"  Words: {results['word_count']}")
    print(f"  Characters: {results['char_count']}")
    print(f"  Topic: {results['topic']}")
    print(f"  Tone: {results['tone']}")

print("\n✅ Parallel analysis provides comprehensive insights instantly!")
```

**What you learned:**
- Running multiple analyses in parallel
- Combining LLM and non-LLM operations
- Efficient text processing

</details>

---

## **Optional Extra Practice**

### **Challenge: Build a Complete Document Processing Pipeline**

**Task**: Create a chain that processes documents end-to-end
**Expected Output**: Fully processed document with multiple outputs

In [ ]:
# Your code here

<details>
<summary>Solution</summary>

```python
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# **Define output structure**
class DocumentAnalysis(BaseModel):
    title: str = Field(description="Document title")
    summary: str = Field(description="Executive summary")
    key_points: List[str] = Field(description="Main points")
    sentiment: str = Field(description="Overall sentiment")
    recommendations: List[str] = Field(description="Action items")
    category: str = Field(description="Document category")

# **Initialize components**
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
analysis_parser = PydanticOutputParser(pydantic_object=DocumentAnalysis)

# **Create comprehensive analysis prompt**
analysis_prompt = ChatPromptTemplate.from_template(
    """Analyze this document comprehensively:
    
    {document}
    
    {format_instructions}
    """
)

# **Create quality check prompt**
quality_prompt = ChatPromptTemplate.from_template(
    """Rate the quality of this document from 1-10 and explain why:
    {document}
    """
)

# **Create translation prompt**
translation_prompt = ChatPromptTemplate.from_template(
    """Translate the key message of this document to Spanish:
    {document}
    """
)

# **Build the complete pipeline**
document_pipeline = RunnableParallel(
    # Main analysis
    analysis=(
        analysis_prompt 
        | llm 
        | analysis_parser
    ),
    # Quality assessment
    quality=(
        quality_prompt 
        | llm 
        | StrOutputParser()
    ),
    # Translation
    spanish_summary=(
        translation_prompt 
        | llm 
        | StrOutputParser()
    ),
    # Word count
    statistics=RunnableLambda(
        lambda x: {
            "word_count": len(x['document'].split()),
            "char_count": len(x['document']),
            "line_count": len(x['document'].split('\n'))
        }
    )
)

# **Test document**
document = """Strategic Plan 2024

Our company faces both challenges and opportunities in the coming year. 
Market conditions are volatile, but our strong product portfolio positions us well.

Key Priorities:
1. Expand into emerging markets
2. Strengthen digital capabilities
3. Improve operational efficiency

We must act decisively to maintain our competitive advantage while managing risks.
Success will require coordinated efforts across all departments.
"""

# **Process the document**
results = document_pipeline.invoke({
    "document": document,
    "format_instructions": analysis_parser.get_format_instructions()
})

print("📄 Complete Document Processing Pipeline")
print("=" * 60)

# **Display comprehensive results**
analysis = results['analysis']
print(f"\n📊 Document Analysis:")
print(f"  Title: {analysis.title}")
print(f"  Category: {analysis.category}")
print(f"  Sentiment: {analysis.sentiment}")
print(f"\n  Summary: {analysis.summary}")
print(f"\n  Key Points:")
for point in analysis.key_points:
    print(f"    • {point}")
print(f"\n  Recommendations:")
for rec in analysis.recommendations:
    print(f"    → {rec}")

print(f"\n⭐ Quality Assessment:")
print(f"  {results['quality'][:200]}...")

print(f"\n🌍 Spanish Summary:")
print(f"  {results['spanish_summary'][:200]}...")

print(f"\n📈 Document Statistics:")
stats = results['statistics']
print(f"  Words: {stats['word_count']}")
print(f"  Characters: {stats['char_count']}")
print(f"  Lines: {stats['line_count']}")

print("\n✅ Complete pipeline processed document with multiple outputs!")
```

**Pipeline capabilities:**
- Comprehensive document analysis
- Multiple parallel operations
- Structured data extraction
- Multi-language support
- Statistical analysis

</details>

---

## **Summary & Next Steps**

### **What You've Learned**
✅ Building chains with the pipe operator (|) in LCEL  
✅ Connecting prompts, LLMs, and parsers into workflows  
✅ Using output parsers for structured data extraction  
✅ Implementing parallel execution with RunnableParallel  
✅ Creating conditional routing in chains  
✅ Building complex multi-step pipelines  

### **Key Takeaways**
1. **LCEL makes chains readable** - The pipe operator creates clear data flow
2. **Output parsers structure data** - Convert text to JSON, Pydantic, lists
3. **Parallel execution saves time** - Run multiple operations simultaneously
4. **Chains are composable** - Build complex workflows from simple components
5. **Routing enables flexibility** - Different paths for different inputs

### **What's Next?**
In the next notebooks, you'll learn:
- Document loading and processing
- Embeddings and vector stores
- Building complete RAG systems
- Creating AI agents with tools
- Production deployment strategies

### **Resources**
- [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/)
- [Output Parsers Documentation](https://python.langchain.com/docs/modules/model_io/output_parsers/)
- [Runnable Interface](https://python.langchain.com/docs/expression_language/interface)
- [LCEL Cookbook](https://python.langchain.com/docs/expression_language/cookbook/)

---

🎉 **Congratulations!** You've mastered chains and LCEL! You can now build sophisticated LLM workflows and processing pipelines.